In [1]:
from transformers import DistilBertModel, DistilBertTokenizer, LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [2]:
import pandas as pd
import numpy as np
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer 
import nltk as nltk
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import string
from nltk.tokenize import word_tokenize
from transformers import BertForMaskedLM,BertConfig,BertTokenizer

In [3]:
df=pd.read_excel('data_sizing_bin1.xlsx')
df=df.drop(['Unnamed: 0',
 'Unnamed: 0.1',
 'TaskID',
'BudgetLineID',
 'ProjectID',
 'RegionID','CurrentProcess',
 'PCPNumber',
 'CR',
 'PCPType','TaskStatus',
 'DescopeReason','OOM',
 'OOMStr', 'CategoryKeyword',
 'LateUATJustification',
 'OOMForumReview',
 'OOMForumApproved',
 'ExpiryFlag',
 'DateOpen',
 'OpenYear',
 'AssessmentMode',
 'OOMForumComments',
 'OOMComments',
 'AssessmentInfoComments',
 'PCPComments','PCPCategory',
 'Itemization',
 'UpdatedDate','Sizing_cpy','UATDuration','AssessmentCategory','DomainID','AreaID','FundingSolution','totalwords_RequirementDescription','Sizing'],axis=1)
cname=list(df.columns)
df=df.reset_index(drop = True)
df['Sizing_bins']=df['Sizing_bins'].replace(['Very Small','Small','Medium','Large'], [0,1,2,3])
#df=df.drop(['Sizing_bins'],axis=1)
print('\n')
print('Null Value list \n',df.isnull().sum())
print('\n') 
print(df.info())



Null Value list 
 Title                        0
Domain                       0
Area                      2444
BudgetLine                1728
ProjectName                  0
ProjectDescription           0
Region                    1441
RequirementDescription      71
SolutionSummary           3994
SolutionDescription       3790
SolutionAssumption        6955
ProductName               3093
RecommendedPricing         264
Sizing_bins               3765
dtype: int64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11113 entries, 0 to 11112
Data columns (total 14 columns):
Title                     11113 non-null object
Domain                    11113 non-null object
Area                      8669 non-null object
BudgetLine                9385 non-null object
ProjectName               11113 non-null object
ProjectDescription        11113 non-null object
Region                    9672 non-null object
RequirementDescription    11042 non-null object
SolutionSummary           7119 non-null o

In [4]:
df=df.astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11113 entries, 0 to 11112
Data columns (total 14 columns):
Title                     11113 non-null object
Domain                    11113 non-null object
Area                      11113 non-null object
BudgetLine                11113 non-null object
ProjectName               11113 non-null object
ProjectDescription        11113 non-null object
Region                    11113 non-null object
RequirementDescription    11113 non-null object
SolutionSummary           11113 non-null object
SolutionDescription       11113 non-null object
SolutionAssumption        11113 non-null object
ProductName               11113 non-null object
RecommendedPricing        11113 non-null object
Sizing_bins               11113 non-null object
dtypes: object(14)
memory usage: 1.2+ MB


In [5]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
REMOVE_NUMBERS =re.compile("[\d+]")
STOPWORDS = set(stopwords.words('english'))


def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = REMOVE_NUMBERS.sub('',text) # replace REMOVE_NUMBERS by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)# delete stopwors from text
    #text = ' '.join(word for word in text.split() if word not in common_words)
    return text

In [6]:
cname=list(df.columns)
for i in cname:
    df[i]=df[i].apply(clean_text)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:333: MarkupResemblesLocatorWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/agile/browse/HITPP-1238" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/agile/browse/HITPP-1239" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  Mar

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/confluence/display/PDA/ST+89616+BR%3A+TKT%3A+Revalidation+control+for+name+ID+code+and+associated+condition" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/confluence/display/PDA/ST+89671+CI%3A+TKT%3A+Enhancement+of+Revalidation+Function" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResembl

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/confluence/display/MERTPC/PCP%2315491994+-+Hotel+chain+provider+in+the+Teaser+Criteria" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/confluence/display/RDAIRTDFBTF/Feasibility+Study+-+CP+16075543" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.am

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/confluence/display/RDAIRTDFBTF/13484525%3A+Feasibility+study" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/confluence/display/ARDWSOL/SWA%3A0754%3AARD%3AAdd+SGA+Banner+in+ARD+shopping+and+availability+screens" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/confluence/display/HITREPORTING/106045+%5BCLARIFIED%5D+-+Migration+Status+Report" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/confluence/pages/viewpage.action?pageId=913927071" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/confluenc

In [7]:
corpus_df = df['Title']+ " " + df['Domain'] + " " + df['Area'] + " " + df['BudgetLine'] + " " + df['ProjectName'] + " " + df['ProjectDescription'] +" "+ df['Region'] + " " +df['RequirementDescription'] + df['SolutionSummary'] +' '+df['SolutionDescription']+' '+df['SolutionAssumption']+' '+df['ProductName']+' '+df['RecommendedPricing']

In [8]:
corpus_df = corpus_df.str.split(' ')

In [9]:
corpus_df

0        [time, spent, reporting, airline, bespoke, cp,...
1        [time, spent, reporting, airline, bespoke, cp,...
2        [svecom, cs, atc, invol, online, dho, ecom, na...
3        [afkl, inv, group, block, seating, inv, seat, ...
4        [af, kl, inv, group, zone, inv, seat, regional...
                               ...                        
11108    [acpcpdcsprint, nd, atb, receipt, fop, debit, ...
11109    [ozwcicredit, card, payment, wci, flow, dcs, n...
11110    [brotherbr, ftp, accounts, password, change, t...
11111    [namesizetypemodifiedattached, cp___branded_fa...
11112    [dcs_actcm, activation, sn, crew, printing, ba...
Length: 11113, dtype: object

In [10]:
cleaned = []

for text in corpus_df:
    text = [x.strip(string.punctuation) for x in text]
    cleaned.append(text)

cleaned[0]

text_spam = [" ".join(text) for text in cleaned]
final_text = " ".join(text_spam)
final_text[:500]

'time spent reporting airline bespoke cp detailed assessment srb gui airline private development  amadeus nan time spent reporting airline bespoke cp detailed assessmenttime spent reporting airline bespoke cp detailed assessment task automatically created importing cr winaproach time spent reporting airline bespoke cp detailed assessment altea time rptg assessment time spent reporting airline bespoke cp oom assessment srb oui airline private development  amadeus nan time spent reporting airline b'

In [11]:
tok_final_text=word_tokenize(final_text)

In [12]:
from collections import Counter
word_counts = Counter(tok_final_text)
words = list(word_counts.most_common(5000))
words.pop(0)

('nan', 19725)

In [13]:
words

[('sold', 18811),
 ('added', 15656),
 ('new', 11728),
 ('airlines', 11027),
 ('cp', 11021),
 ('fee', 10472),
 ('fees', 10027),
 ('customer', 9845),
 ('requesting', 9799),
 ('airline', 9711),
 ('change', 7881),
 ('dcs', 7273),
 ('development', 6990),
 ('flight', 6875),
 ('amadeus', 6277),
 ('solution', 6261),
 ('created', 5987),
 ('code', 5974),
 ('task', 5519),
 ('automatically', 5214),
 ('pnr', 5153),
 ('document', 4886),
 ('apac', 4764),
 ('altea', 4607),
 ('importing', 4526),
 ('data', 4393),
 ('number', 4257),
 ('n', 4220),
 ('time', 4197),
 ('cm', 4152),
 ('information', 4140),
 ('would', 4061),
 ('res', 3988),
 ('update', 3752),
 ('display', 3701),
 ('one', 3673),
 ('addedusage', 3641),
 ('airlinesmaintenance', 3596),
 ('date', 3582),
 ('fare', 3569),
 ('seat', 3475),
 ('use', 3469),
 ('process', 3450),
 ('air', 3433),
 ('need', 3424),
 ('rule', 3410),
 ('boarding', 3392),
 ('customerhow', 3380),
 ('check', 3357),
 ('add', 3346),
 ('private', 3220),
 ('scope', 3194),
 ('requireme

In [14]:
import torch
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")
print(len(tokenizer))  # 28996
tokenizer.add_tokens(words)
print(len(tokenizer)) 
model.resize_token_embeddings(len(tokenizer)) 

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


30522
35521


Embedding(35521, 768)

In [15]:
%%time
from transformers import TextDataset

dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="corpus.txt",
    block_size=64,
)

CPU times: user 54.4 s, sys: 506 ms, total: 54.9 s
Wall time: 55.2 s


In [16]:
# Batching + preprocessing for MLM
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [17]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="~/cloudfiles/code/Users/jimharrington.johnselvynnallathambi",
    overwrite_output_dir=True,
    num_train_epochs=20,
    per_gpu_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/transformers/trainer.py:256: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead.
  FutureWarning,


In [18]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


{'loss': 5.31205126953125, 'learning_rate': 4.9825662482566246e-05, 'epoch': 0.0697350069735007, 'total_flos': 87512924928000, 'step': 500}
{'loss': 4.82885693359375, 'learning_rate': 4.9651324965132504e-05, 'epoch': 0.1394700139470014, 'total_flos': 175025849856000, 'step': 1000}
{'loss': 4.64662890625, 'learning_rate': 4.947698744769875e-05, 'epoch': 0.20920502092050208, 'total_flos': 262538774784000, 'step': 1500}
{'loss': 4.61073046875, 'learning_rate': 4.930264993026499e-05, 'epoch': 0.2789400278940028, 'total_flos': 350051699712000, 'step': 2000}
{'loss': 4.47573046875, 'learning_rate': 4.912831241283124e-05, 'epoch': 0.3486750348675035, 'total_flos': 437564624640000, 'step': 2500}
{'loss': 4.368923828125, 'learning_rate': 4.895397489539749e-05, 'epoch': 0.41841004184100417, 'total_flos': 525077549568000, 'step': 3000}
{'loss': 4.373060546875, 'learning_rate': 4.877963737796374e-05, 'epoch': 0.4881450488145049, 'total_flos': 612590474496000, 'step': 3500}
{'loss': 4.356376953125,

{'loss': 4.02865234375, 'learning_rate': 4.738493723849373e-05, 'epoch': 1.0460251046025104, 'total_flos': 1312693873920000, 'step': 7500}
{'loss': 3.989609375, 'learning_rate': 4.7210599721059973e-05, 'epoch': 1.1157601115760112, 'total_flos': 1400206798848000, 'step': 8000}
{'loss': 3.9699375, 'learning_rate': 4.7036262203626224e-05, 'epoch': 1.185495118549512, 'total_flos': 1487719723776000, 'step': 8500}
{'loss': 4.012328125, 'learning_rate': 4.686192468619247e-05, 'epoch': 1.2552301255230125, 'total_flos': 1575232648704000, 'step': 9000}
{'loss': 3.941671875, 'learning_rate': 4.668758716875872e-05, 'epoch': 1.3249651324965133, 'total_flos': 1662745573632000, 'step': 9500}
{'loss': 3.96065625, 'learning_rate': 4.651324965132497e-05, 'epoch': 1.394700139470014, 'total_flos': 1750258498560000, 'step': 10000}
{'loss': 3.9030234375, 'learning_rate': 4.6338912133891213e-05, 'epoch': 1.4644351464435146, 'total_flos': 1837771423488000, 'step': 10500}
{'loss': 3.9408046875, 'learning_rate'

{'loss': 3.793375, 'learning_rate': 4.4944211994421206e-05, 'epoch': 2.0223152022315203, 'total_flos': 2537874822912000, 'step': 14500}
{'loss': 3.7606796875, 'learning_rate': 4.476987447698745e-05, 'epoch': 2.092050209205021, 'total_flos': 2625387747840000, 'step': 15000}
{'loss': 3.773015625, 'learning_rate': 4.4595536959553694e-05, 'epoch': 2.161785216178522, 'total_flos': 2712900672768000, 'step': 15500}
{'loss': 3.741859375, 'learning_rate': 4.442119944211995e-05, 'epoch': 2.2315202231520224, 'total_flos': 2800413597696000, 'step': 16000}
{'loss': 3.757171875, 'learning_rate': 4.4246861924686195e-05, 'epoch': 2.301255230125523, 'total_flos': 2887926522624000, 'step': 16500}
{'loss': 3.72765625, 'learning_rate': 4.407252440725244e-05, 'epoch': 2.370990237099024, 'total_flos': 2975439447552000, 'step': 17000}
{'loss': 3.69540625, 'learning_rate': 4.389818688981869e-05, 'epoch': 2.4407252440725244, 'total_flos': 3062952372480000, 'step': 17500}
{'loss': 3.715859375, 'learning_rate': 

{'loss': 3.607515625, 'learning_rate': 4.2329149232914926e-05, 'epoch': 3.0683403068340307, 'total_flos': 3850568696832000, 'step': 22000}
{'loss': 3.572328125, 'learning_rate': 4.215481171548118e-05, 'epoch': 3.1380753138075312, 'total_flos': 3938081621760000, 'step': 22500}
{'loss': 3.6033125, 'learning_rate': 4.198047419804742e-05, 'epoch': 3.2078103207810322, 'total_flos': 4025594546688000, 'step': 23000}
{'loss': 3.576046875, 'learning_rate': 4.180613668061367e-05, 'epoch': 3.2775453277545328, 'total_flos': 4113107471616000, 'step': 23500}
{'loss': 3.51809375, 'learning_rate': 4.1631799163179915e-05, 'epoch': 3.3472803347280333, 'total_flos': 4200620396544000, 'step': 24000}
{'loss': 3.571875, 'learning_rate': 4.1457461645746166e-05, 'epoch': 3.4170153417015343, 'total_flos': 4288133321472000, 'step': 24500}
{'loss': 3.557171875, 'learning_rate': 4.128312412831242e-05, 'epoch': 3.486750348675035, 'total_flos': 4375646246400000, 'step': 25000}
{'loss': 3.58646875, 'learning_rate': 

...

{'loss': 2.7176875, 'learning_rate': 9.9023709902371e-06, 'epoch': 16.03905160390516, 'total_flos': 20127972733440000, 'step': 115000}
{'loss': 2.6479375, 'learning_rate': 9.728033472803348e-06, 'epoch': 16.10878661087866, 'total_flos': 20215485658368000, 'step': 115500}
{'loss': 2.6415, 'learning_rate': 9.553695955369596e-06, 'epoch': 16.178521617852162, 'total_flos': 20302998583296000, 'step': 116000}
{'loss': 2.5889375, 'learning_rate': 9.379358437935843e-06, 'epoch': 16.24825662482566, 'total_flos': 20390511508224000, 'step': 116500}
{'loss': 2.7016875, 'learning_rate': 9.205020920502094e-06, 'epoch': 16.317991631799163, 'total_flos': 20478024433152000, 'step': 117000}
{'loss': 2.633125, 'learning_rate': 9.030683403068341e-06, 'epoch': 16.387726638772662, 'total_flos': 20565537358080000, 'step': 117500}
{'loss': 2.7150625, 'learning_rate': 8.856345885634588e-06, 'epoch': 16.457461645746164, 'total_flos': 20653050283008000, 'step': 118000}
{'loss': 2.65525, 'learning_rate': 8.682008

{'loss': 2.6475, 'learning_rate': 7.461645746164575e-06, 'epoch': 17.01534170153417, 'total_flos': 21353153682432000, 'step': 122000}
{'loss': 2.6516875, 'learning_rate': 7.287308228730823e-06, 'epoch': 17.08507670850767, 'total_flos': 21440666607360000, 'step': 122500}
{'loss': 2.626625, 'learning_rate': 7.112970711297072e-06, 'epoch': 17.15481171548117, 'total_flos': 21528179532288000, 'step': 123000}
{'loss': 2.6019375, 'learning_rate': 6.93863319386332e-06, 'epoch': 17.224546722454672, 'total_flos': 21615692457216000, 'step': 123500}
{'loss': 2.6725, 'learning_rate': 6.7642956764295674e-06, 'epoch': 17.294281729428175, 'total_flos': 21703205382144000, 'step': 124000}
{'loss': 2.567625, 'learning_rate': 6.5899581589958165e-06, 'epoch': 17.364016736401673, 'total_flos': 21790718307072000, 'step': 124500}
{'loss': 2.6620625, 'learning_rate': 6.415620641562065e-06, 'epoch': 17.433751743375176, 'total_flos': 21878231232000000, 'step': 125000}
{'loss': 2.6303125, 'learning_rate': 6.24128

{'loss': 2.633625, 'learning_rate': 4.846582984658299e-06, 'epoch': 18.061366806136682, 'total_flos': 22665847556352000, 'step': 129500}
{'loss': 2.621125, 'learning_rate': 4.6722454672245465e-06, 'epoch': 18.13110181311018, 'total_flos': 22753360481280000, 'step': 130000}
{'loss': 2.6181875, 'learning_rate': 4.497907949790795e-06, 'epoch': 18.200836820083683, 'total_flos': 22840873406208000, 'step': 130500}
{'loss': 2.6068125, 'learning_rate': 4.323570432357044e-06, 'epoch': 18.270571827057182, 'total_flos': 22928386331136000, 'step': 131000}
{'loss': 2.614125, 'learning_rate': 4.149232914923291e-06, 'epoch': 18.340306834030685, 'total_flos': 23015899256064000, 'step': 131500}
{'loss': 2.6053125, 'learning_rate': 3.97489539748954e-06, 'epoch': 18.410041841004183, 'total_flos': 23103412180992000, 'step': 132000}
{'loss': 2.511375, 'learning_rate': 3.8005578800557883e-06, 'epoch': 18.479776847977686, 'total_flos': 23190925105920000, 'step': 132500}
{'loss': 2.5771875, 'learning_rate': 3

{'loss': 2.5181875, 'learning_rate': 2.4058577405857742e-06, 'epoch': 19.03765690376569, 'total_flos': 23891028505344000, 'step': 136500}
{'loss': 2.5429375, 'learning_rate': 2.2315202231520224e-06, 'epoch': 19.107391910739192, 'total_flos': 23978541430272000, 'step': 137000}
{'loss': 2.5803125, 'learning_rate': 2.057182705718271e-06, 'epoch': 19.17712691771269, 'total_flos': 24066054355200000, 'step': 137500}
{'loss': 2.521375, 'learning_rate': 1.882845188284519e-06, 'epoch': 19.246861924686193, 'total_flos': 24153567280128000, 'step': 138000}
{'loss': 2.55575, 'learning_rate': 1.7085076708507672e-06, 'epoch': 19.316596931659692, 'total_flos': 24241080205056000, 'step': 138500}
{'loss': 2.5885625, 'learning_rate': 1.5341701534170154e-06, 'epoch': 19.386331938633194, 'total_flos': 24328593129984000, 'step': 139000}
{'loss': 2.5526875, 'learning_rate': 1.3598326359832638e-06, 'epoch': 19.456066945606693, 'total_flos': 24416106054912000, 'step': 139500}
{'loss': 2.537875, 'learning_rate'

TrainOutput(global_step=143400, training_loss=3.116657078103208)

In [19]:
trainer.save_model("amd_bert-base-uncased")